<a href="https://colab.research.google.com/github/PSriram2517/FMML_IAB_23B21A4538/blob/main/Copy_of_FMML_M1L2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Machine Learning terms and metrics

FMML Module 1, Lab 2

In this lab, we will show a part of the ML pipeline by using the California Housing dataset. There are 20640 samples, each with 8 attributes like income of the block, age of the houses per district etc. The task is to predict the cost of the houses per district. We will use the scikit-learn library to load the data and perform some basic data preprocessing and model training. We will also show how to evaluate the model using some common metrics, split the data into training and testing sets, and use cross-validation to get a better estimate of the model's performance.

In [ ]:
import numpy as np
from sklearn import datasets
import matplotlib.pyplot as plt

rng = np.random.default_rng(seed=42)

In [ ]:
dataset = datasets.fetch_california_housing()
# Dataset description
print(dataset.DESCR)

.. _california_housing_dataset:

California Housing dataset
--------------------------

**Data Set Characteristics:**

    :Number of Instances: 20640

    :Number of Attributes: 8 numeric, predictive attributes and the target

    :Attribute Information:
        - MedInc        median income in block group
        - HouseAge      median house age in block group
        - AveRooms      average number of rooms per household
        - AveBedrms     average number of bedrooms per household
        - Population    block group population
        - AveOccup      average number of household members
        - Latitude      block group latitude
        - Longitude     block group longitude

    :Missing Attribute Values: None

This dataset was obtained from the StatLib repository.
https://www.dcc.fc.up.pt/~ltorgo/Regression/cal_housing.html

The target variable is the median house value for California districts,
expressed in hundreds of thousands of dollars ($100,000).

This dataset was derived

Given below are the list of target values. These correspond to the house value derived considering all the 8 input features and are continuous values. We should use regression models to predict these values but we will start with a simple classification model for the sake of simplicity. We need to just round off the values to the nearest integer and use a classification model to predict the house value.

In [ ]:
print("Orignal target values:", dataset.target)

dataset.target = dataset.target.astype(int)

print("Target values after conversion:", dataset.target)
print("Input variables shape:", dataset.data.shape)
print("Output variables shape:", dataset.target.shape)

Orignal target values: [4.526 3.585 3.521 ... 0.923 0.847 0.894]
Target values after conversion: [4 3 3 ... 0 0 0]
Input variables shape: (20640, 8)
Output variables shape: (20640,)


The simplest model to use for classification is the K-Nearest Neighbors model. We will use this model to predict the house value with a K value of 1. We will also use the accuracy metric to evaluate the model.

In [ ]:
def NN1(traindata, trainlabel, query):
    """
    This function takes in the training data, training labels and a query point
    and returns the predicted label for the query point using the nearest neighbour algorithm

    traindata: numpy array of shape (n,d) where n is the number of samples and d is the number of features
    trainlabel: numpy array of shape (n,) where n is the number of samples
    query: numpy array of shape (d,) where d is the number of features

    returns: the predicted label for the query point which is the label of the training data which is closest to the query point
    """
    diff = (
        traindata - query
    )  # find the difference between features. Numpy automatically takes care of the size here
    sq = diff * diff  # square the differences
    dist = sq.sum(1)  # add up the squares
    label = trainlabel[np.argmin(dist)]
    return label


def NN(traindata, trainlabel, testdata):
    """
    This function takes in the training data, training labels and test data
    and returns the predicted labels for the test data using the nearest neighbour algorithm

    traindata: numpy array of shape (n,d) where n is the number of samples and d is the number of features
    trainlabel: numpy array of shape (n,) where n is the number of samples
    testdata: numpy array of shape (m,d) where m is the number of test samples and d is the number of features

    returns: the predicted labels for the test data which is the label of the training data which is closest to each test point
    """
    predlabel = np.array([NN1(traindata, trainlabel, i) for i in testdata])
    return predlabel

We will also define a 'random classifier', which randomly allots labels to each sample

In [ ]:
def RandomClassifier(traindata, trainlabel, testdata):
    """
    This function takes in the training data, training labels and test data
    and returns the predicted labels for the test data using the random classifier algorithm

    In reality, we don't need these arguments but we are passing them to keep the function signature consistent with other classifiers

    traindata: numpy array of shape (n,d) where n is the number of samples and d is the number of features
    trainlabel: numpy array of shape (n,) where n is the number of samples
    testdata: numpy array of shape (m,d) where m is the number of test samples and d is the number of features

    returns: the predicted labels for the test data which is a random label from the training data
    """

    classes = np.unique(trainlabel)
    rints = rng.integers(low=0, high=len(classes), size=len(testdata))
    predlabel = classes[rints]
    return predlabel

We need a metric to evaluate the performance of the model. Let us define a metric 'Accuracy' to see how good our learning algorithm is. Accuracy is the ratio of the number of correctly classified samples to the total number of samples. The higher the accuracy, the better the algorithm. We will use the accuracy metric to evaluate and compate the performance of the K-Nearest Neighbors model and the random classifier.

In [ ]:
def Accuracy(gtlabel, predlabel):
    """
    This function takes in the ground-truth labels and predicted labels
    and returns the accuracy of the classifier

    gtlabel: numpy array of shape (n,) where n is the number of samples
    predlabel: numpy array of shape (n,) where n is the number of samples

    returns: the accuracy of the classifier which is the number of correct predictions divided by the total number of predictions
    """
    assert len(gtlabel) == len(
        predlabel
    ), "Length of the ground-truth labels and predicted labels should be the same"
    correct = (
        gtlabel == predlabel
    ).sum()  # count the number of times the groundtruth label is equal to the predicted label.
    return correct / len(gtlabel)

Let us make a function to split the dataset with the desired probability. We will use this function to split the dataset into training and testing sets. We will use the training set to train the model and the testing set to evaluate the model.

In [ ]:
def split(data, label, percent):
    # generate a random number for each sample
    rnd = rng.random(len(label))
    split1 = rnd < percent
    split2 = rnd >= percent

    split1data = data[split1, :]
    split1label = label[split1]
    split2data = data[split2, :]
    split2label = label[split2]
    return split1data, split1label, split2data, split2label

We will reserve 20% of our dataset as the test set. We will not change this portion throughout our experiments

In [ ]:
testdata, testlabel, alltraindata, alltrainlabel = split(
    dataset.data, dataset.target, 20 / 100
)
print("Number of test samples:", len(testlabel))
print("Number of train samples:", len(alltrainlabel))
print("Percent of test data:", len(testlabel) * 100 / len(dataset.target), "%")

Number of test samples: 4144
Number of train samples: 16496
Percent of test data: 20.07751937984496 %


## Experiments with splits

Let us reserve some of our train data as a validation set

In [ ]:
traindata, trainlabel, valdata, vallabel = split(
    alltraindata, alltrainlabel, 75 / 100)

What is the accuracy of our classifiers on the train dataset?

In [ ]:
trainpred = NN(traindata, trainlabel, traindata)
trainAccuracy = Accuracy(trainlabel, trainpred)
print("Training accuracy using nearest neighbour algorithm:", trainAccuracy*100, "%")

trainpred = RandomClassifier(traindata, trainlabel, traindata)
trainAccuracy = Accuracy(trainlabel, trainpred)
print("Training accuracy using random classifier: ", trainAccuracy*100, "%")

Training accuracy using nearest neighbour algorithm: 100.0 %
Training accuracy using random classifier:  16.4375808538163 %


For nearest neighbour, the train accuracy is always 1. The accuracy of the random classifier is close to 1/(number of classes) which is 0.1666 in our case. This is because the random classifier randomly assigns a label to each sample and the probability of assigning the correct label is 1/(number of classes). Let us predict the labels for our validation set and get the accuracy. This accuracy is a good estimate of the accuracy of our model on unseen data.

In [ ]:
valpred = NN(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy using nearest neighbour algorithm:", valAccuracy*100, "%")


valpred = RandomClassifier(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy using random classifier:", valAccuracy*100, "%")

Validation accuracy using nearest neighbour algorithm: 34.10852713178294 %
Validation accuracy using random classifier: 16.884689922480618 %


Validation accuracy of nearest neighbour is considerably less than its train accuracy while the validation accuracy of random classifier is the same. However, the validation accuracy of nearest neighbour is twice that of the random classifier. Now let us try another random split and check the validation accuracy. We will see that the validation accuracy changes with the split. This is because the validation set is small and the accuracy is highly dependent on the samples in the validation set. We can get a better estimate of the accuracy by using cross-validation.

In [ ]:
traindata, trainlabel, valdata, vallabel = split(
    alltraindata, alltrainlabel, 75 / 100)
valpred = NN(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy using nearest neighbour algorithm:", valAccuracy*100, "%")

Validation accuracy using nearest neighbour algorithm: 34.048257372654156 %


You can run the above cell multiple times to try with different random splits.
We notice that the accuracy is different for each run, but close together.

Now let us compare it with the accuracy we get on the test dataset.

In [ ]:
testpred = NN(alltraindata, alltrainlabel, testdata)
testAccuracy = Accuracy(testlabel, testpred)

print("Test accuracy:", testAccuracy*100, "%")

Test accuracy: 34.91795366795367 %


### Try it out for yourself and answer:
1. How is the accuracy of the validation set affected if we increase the percentage of validation set? What happens when we reduce it?
2. How does the size of the train and validation set affect how well we can predict the accuracy on the test set using the validation set?
3. What do you think is a good percentage to reserve for the validation set so that thest two factors are balanced?

Answer for both nearest neighbour and random classifier. You can note down the values for your experiments and plot a graph using  <a href=https://matplotlib.org/stable/gallery/lines_bars_and_markers/step_demo.html#sphx-glr-gallery-lines-bars-and-markers-step-demo-py>plt.plot<href>. Check also for extreme values for splits, like 99.9% or 0.1%

> Exercise: Try to implement a 3 nearest neighbour classifier and compare the accuracy of the 1 nearest neighbour classifier and the 3 nearest neighbour classifier on the test dataset. You can use the KNeighborsClassifier class from the scikit-learn library to implement the K-Nearest Neighbors model. You can set the number of neighbors using the n_neighbors parameter. You can also use the accuracy_score function from the scikit-learn library to calculate the accuracy of the model.

## Multiple Splits

One way to get more accurate estimates for the test accuracy is by using <b>cross-validation</b>. Here, we will try a simple version, where we do multiple train/val splits and take the average of validation accuracies as the test accuracy estimation. Here is a function for doing this. Note that this function will take a long time to execute. You can reduce the number of splits to make it faster.

In [ ]:
def AverageAccuracy(alldata, alllabel, splitpercent, iterations, classifier=NN):
    """
    This function takes in the data, labels, split percentage, number of iterations and classifier function
    and returns the average accuracy of the classifier

    alldata: numpy array of shape (n,d) where n is the number of samples and d is the number of features
    alllabel: numpy array of shape (n,) where n is the number of samples
    splitpercent: float which is the percentage of data to be used for training
    iterations: int which is the number of iterations to run the classifier
    classifier: function which is the classifier function to be used

    returns: the average accuracy of the classifier
    """
    accuracy = 0
    for ii in range(iterations):
        traindata, trainlabel, valdata, vallabel = split(
            alldata, alllabel, splitpercent
        )
        valpred = classifier(traindata, trainlabel, valdata)
        accuracy += Accuracy(vallabel, valpred)
    return accuracy / iterations  # average of all accuracies

In [ ]:
avg_acc = AverageAccuracy(alltraindata, alltrainlabel, 75 / 100, 10, classifier=NN)
print("Average validation accuracy:", avg_acc*100, "%")
testpred = NN(alltraindata, alltrainlabel, testdata)

print("Test accuracy:", Accuracy(testlabel, testpred)*100, "%")

Average validation accuracy: 33.58463539517022 %
Test accuracy: 34.91795366795367 %


This is a very simple way of doing cross-validation. There are many well-known algorithms for cross-validation, like k-fold cross-validation, leave-one-out etc. This will be covered in detail in a later module. For more information about cross-validation, check <a href=https://en.wikipedia.org/wiki/Cross-validation_(statistics)>Cross-validatioin (Wikipedia)</a>

### Questions
1. Does averaging the validation accuracy across multiple splits give more consistent results?
2. Does it give more accurate estimate of test accuracy?
3. What is the effect of the number of iterations on the estimate? Do we get a better estimate with higher iterations?
4. Consider the results you got for the previous questions. Can we deal with a very small train dataset or validation dataset by increasing the iterations?


> Exercise: How does the accuracy of the 3 nearest neighbour classifier change with the number of splits? How is it affected by the split size? Compare the results with the 1 nearest neighbour classifier.

1.The "best" combination of Python features depends on what you're trying to achieve. Below are some optimal combinations tailored to different domains:

### 1. **Data Science & Machine Learning**
   - **Jupyter Notebooks**: For interactive data exploration and visualization.
   - **Pandas + NumPy**: For efficient data manipulation and numerical computations.
   - **Matplotlib + Seaborn**: For creating detailed and aesthetic data visualizations.
   - **Scikit-Learn**: For implementing machine learning algorithms and models.
   - **TensorFlow or PyTorch**: For deep learning and neural networks.

   **Why this combination?** It covers everything from data exploration and cleaning to model building, training, and visualization, making it an all-encompassing setup for data science.

### 2. **Web Development**
   - **Django or Flask**: Django for a full-fledged framework with built-in admin, ORM, and authentication; Flask for a lightweight, more flexible option.
   - **SQLAlchemy**: For database management using an ORM (Object-Relational Mapping).
   - **Jinja2**: For rendering dynamic HTML templates (used with Flask and optionally with Django).
   - **Gunicorn or Uvicorn**: For deploying your web app in a production environment. Gunicorn is suited for Django, while Uvicorn works well with FastAPI and Flask (when using async features).

   **Why this combination?** It provides a solid foundation for building, managing, and deploying web applications, whether you need a robust framework (Django) or something more modular (Flask).

### 3. **Automation & Scripting**
   - **Requests**: For making HTTP requests to interact with web APIs.
   - **BeautifulSoup + lxml**: For web scraping and parsing HTML or XML.
   - **Selenium**: For automating web browsers, especially when dealing with JavaScript-heavy pages.
   - **argparse**: For handling command-line arguments in your scripts.
   - **Subprocess**: For executing shell commands and managing processes from within Python.

   **Why this combination?** It provides powerful tools for automating tasks, from simple scripts to complex web scraping and interaction.

### 4. **Game Development**
   - **Pygame**: For developing 2D games and multimedia applications.
   - **PyOpenGL**: For more advanced graphics and 3D rendering.
   - **Panda3D**: If you need a full-fledged 3D game engine.

   **Why this combination?** These libraries provide a strong foundation for both 2D and 3D game development, depending on your needs.

### 5. **GUI Development**
   - **Tkinter**: For creating simple, cross-platform desktop applications.
   - **PyQt or PySide**: For more complex and modern-looking desktop apps.
   - **Kivy**: For developing multi-touch applications, especially for mobile platforms.

   **Why this combination?** It offers a range of tools from simple to advanced, depending on the complexity and platform requirements of your application.

### 6. **DevOps & System Administration**
   - **Ansible + Paramiko**: For automating server configuration and management.
   - **Fabric**: For SSH command execution and deployment automation.
   - **Docker-Py**: For managing Docker containers programmatically.
   - **Psutil**: For monitoring and interacting with system processes and hardware.

   **Why this combination?** It covers various aspects of automation and system management, from server configuration to container orchestration.

Each of these combinations is optimized for a specific type of project, making them the "best" choice depending on what you’re working on.

2.If you’re asking whether certain Python tools or techniques provide a more accurate estimate of test accuracy in machine learning, the answer depends on how you use them. Here’s how to achieve more accurate estimates of test accuracy:

### 1. **Cross-Validation (with Scikit-Learn)**
   - **K-Fold Cross-Validation**: Splits the dataset into \( k \) subsets, trains the model on \( k-1 \) subsets, and tests it on the remaining one. This process is repeated \( k \) times, and the results are averaged.
   - **Stratified K-Fold**: A variation of K-Fold that maintains the distribution of classes in each fold, leading to more reliable results in imbalanced datasets.

   **Why it improves accuracy estimation?** Cross-validation provides a better estimate of model performance because it reduces the variance associated with a single train-test split.

### 2. **Ensemble Methods**
   - **Bagging (e.g., Random Forest)**: Aggregates predictions from multiple models trained on different subsets of the data.
   - **Boosting (e.g., Gradient Boosting, XGBoost)**: Sequentially builds models, each correcting errors made by the previous ones.

   **Why it improves accuracy estimation?** Ensembles typically provide more stable and robust predictions, leading to a more accurate reflection of the model's generalization ability.

### 3. **Hyperparameter Tuning (with Grid Search or Random Search)**
   - **GridSearchCV/RandomizedSearchCV**: Performs an exhaustive or random search over a specified parameter grid to find the best model configuration.

   **Why it improves accuracy estimation?** Properly tuned hyperparameters ensure that the model is neither underfitting nor overfitting, leading to a more accurate estimate of test accuracy.

### 4. **Evaluation Metrics Beyond Accuracy**
   - **Precision, Recall, F1-Score**: Especially in cases of class imbalance, these metrics provide a more nuanced view of model performance.
   - **ROC-AUC**: Evaluates the trade-off between true positive and false positive rates, offering a threshold-independent performance measure.

   **Why it improves accuracy estimation?** Accuracy alone can be misleading, especially in imbalanced datasets. These metrics give a more comprehensive view of model performance.

### 5. **Learning Curves**
   - **Plotting Learning Curves**: Helps to visualize the training and validation accuracy over time and data size, giving insight into overfitting or underfitting.

   **Why it improves accuracy estimation?** It provides a better understanding of how your model's performance changes with more data or iterations, helping to identify and correct issues.

### 6. **Confidence Intervals for Accuracy**
   - **Bootstrapping**: Allows you to estimate the confidence interval of the test accuracy by repeatedly resampling the dataset and calculating the accuracy.

   **Why it improves accuracy estimation?** Confidence intervals provide a range in which the true test accuracy is likely to fall, rather than a single point estimate.

### Conclusion
Using advanced techniques like cross-validation, hyperparameter tuning, and ensemble methods in Python can lead to a more accurate and reliable estimate of test accuracy. Additionally, considering alternative metrics and statistical methods, like confidence intervals, gives a more nuanced understanding of model performance.

3.The number of iterations in various machine learning processes can significantly impact the accuracy and reliability of the estimate. Here’s how the number of iterations affects different scenarios:

### 1. **Cross-Validation**
   - **Effect of More Folds**: When using techniques like K-Fold Cross-Validation, increasing the number of folds (i.e., more iterations) can lead to a more accurate and stable estimate of the model's performance.
   - **Why More Iterations Help**: More folds mean the model is evaluated on more varied subsets of the data, reducing the likelihood of overfitting to a specific train-test split and providing a better estimate of how the model will generalize.

### 2. **Boosting Algorithms (e.g., Gradient Boosting, AdaBoost)**
   - **Effect of More Iterations**: Boosting algorithms typically improve their performance with more iterations, as each new model in the sequence corrects the errors of the previous ones.
   - **Why More Iterations Help (to a Point)**: Initially, more iterations generally improve accuracy as the model becomes better at capturing patterns in the data. However, after a certain point, too many iterations can lead to overfitting, where the model starts to learn the noise in the training data rather than the underlying pattern.

### 3. **Neural Networks & Deep Learning**
   - **Effect of More Epochs**: In neural networks, the number of epochs (complete passes through the training data) is a critical hyperparameter.
   - **Why More Iterations Help (to a Point)**: Initially, more epochs can improve the model’s learning, allowing it to capture complex patterns. However, like boosting, too many epochs can lead to overfitting, where the model performs exceptionally well on training data but poorly on unseen data.

### 4. **Stochastic Gradient Descent (SGD)**
   - **Effect of More Iterations**: In optimization algorithms like SGD, the number of iterations (or steps) determines how thoroughly the algorithm searches for the optimal solution.
   - **Why More Iterations Help**: More iterations can lead to a more accurate convergence towards the minimum of the loss function. However, too many iterations without proper learning rate adjustments can cause the algorithm to oscillate around the minimum or even diverge.

### 5. **Monte Carlo Simulations**
   - **Effect of More Iterations**: Monte Carlo methods rely on repeated random sampling to estimate statistical properties or model performance.
   - **Why More Iterations Help**: More iterations provide a more accurate estimate by reducing the variance of the result, leading to more stable and reliable predictions.

### 6. **Bagging Methods (e.g., Random Forest)**
   - **Effect of More Trees (Iterations)**: Bagging methods like Random Forests improve their performance with more trees (iterations).
   - **Why More Iterations Help**: More trees generally reduce the variance of the model, leading to a more accurate and stable estimate of performance. However, the improvement tends to plateau after a certain number of trees.

### Summary
In general, more iterations often lead to better estimates as they allow the model to explore and learn more from the data, reducing variance and improving accuracy. However, this is true only up to a point:

- **Diminishing Returns**: Beyond a certain threshold, the benefit of additional iterations diminishes, and the model might start overfitting.
- **Overfitting Risk**: More iterations can sometimes lead to overfitting, where the model performs well on training data but poorly on new, unseen data.

To balance this, it’s crucial to monitor performance on a validation set (or through cross-validation) and implement early stopping criteria, regularization techniques, or use ensemble methods to avoid overfitting while maximizing the benefit of more iterations.3.

4.Increasing the number of iterations (whether in cross-validation, boosting, or training epochs) can help improve the performance and stability of a model to some extent, but it does not fully compensate for the limitations of a very small training or validation dataset. Here’s why:

### 1. **Small Training Dataset**
   - **Challenges**:
     - **Lack of Generalization**: A small training dataset may not adequately represent the true distribution of the data, making it harder for the model to generalize well to new, unseen data.
     - **Overfitting Risk**: With limited data, the model is more likely to memorize the training examples rather than learn general patterns, especially as you increase the number of iterations.

   - **Effect of More Iterations**:
     - **Initial Improvement**: Increasing iterations (such as epochs in neural networks or trees in a boosting model) may initially improve the model's ability to capture patterns from the small dataset.
     - **Diminishing Returns**: Beyond a certain point, more iterations can exacerbate overfitting, leading the model to perform worse on unseen data despite improvements on the training set.

   - **Strategies to Mitigate Issues**:
     - **Data Augmentation**: If possible, artificially augment the training data (e.g., by rotating, cropping, or flipping images in computer vision tasks) to create a larger, more diverse training set.
     - **Transfer Learning**: Use a pre-trained model and fine-tune it on the small dataset, leveraging knowledge learned from a larger dataset.
     - **Regularization**: Apply techniques like dropout (in neural networks), L2 regularization, or early stopping to prevent overfitting.

### 2. **Small Validation Dataset**
   - **Challenges**:
     - **Unreliable Performance Metrics**: A small validation set may give an unreliable estimate of the model's performance, as it may not be representative of the general data distribution.
     - **High Variance**: The accuracy and other performance metrics can vary significantly depending on the exact composition of the small validation set.

   - **Effect of More Iterations**:
     - **Initial Stabilization**: Increasing iterations might help stabilize the training process, but it won’t fundamentally solve the problem of an unreliable validation set.
     - **Overfitting to Validation Set**: If the validation set is too small and iterations increase, there's a risk that the model indirectly overfits to the validation data, especially if hyperparameters are tuned based on this set.

   - **Strategies to Mitigate Issues**:
     - **Cross-Validation**: Use techniques like K-Fold or Stratified K-Fold Cross-Validation, where the model is trained and validated multiple times on different subsets of the data, providing a more reliable performance estimate.
     - **Bootstrapping**: Generate multiple validation datasets by sampling with replacement from the available data and average the results to reduce variance in the performance estimate.

### Summary
While increasing the number of iterations can help improve model performance and stability, it **cannot fully compensate for the limitations of a very small training or validation dataset**. The primary issues—such as lack of representativeness, overfitting, and high variance in performance estimates—are intrinsic to the size of the dataset and require more sophisticated techniques to address.

In essence:
- **For small training datasets**: Consider data augmentation, transfer learning, or using models that are less prone to overfitting.
- **For small validation datasets**: Employ cross-validation or bootstrapping to obtain more reliable performance estimates.

Relying solely on more iterations is insufficient and may even worsen the model's performance if not carefully managed..